In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sportsball
from sportsball.nfl import Season, Team, Game
from sportsball.parsing import MasseyParser
from gspread_pandas import Spread, Client

In [3]:
season_2018 = Season(
    year=2018,
    sheettitle='NFL 2018 Expected Wins',
)

In [4]:
season_2018.attach_sagarin_ratings()
season_2018.attach_538_ratings()
season_2018.attach_massey_ratings()
season_2018.attach_scorex_ratings()
season_2018.attach_vegas_ratings()

In [15]:
for game in season_2018.games:
    game.prob_win = game.predict_prob_vegas()

In [18]:
season_2018.simulate()

In [19]:
season_2018.print_divisions()

AFC
  AFC East
    New England    | 11 -  0 -  5 |  69% |  6 |  8
    Miami          |  9 -  0 -  7 |  56% |  0 |  0
    Buffalo        |  5 -  0 - 11 |  31% |  0 | -4
    N.Y. Jets      |  5 -  0 - 11 |  31% | -6 | -6
  AFC North
    Cincinnati     | 12 -  0 -  4 |  75% |  2 |  6
    Baltimore      |  9 -  0 -  7 |  56% |  0 |  2
    Pittsburgh     |  8 -  1 -  7 |  53% | -1 | -1
    Cleveland      |  5 -  1 - 10 |  34% | -1 | -1
  AFC West
    Kansas City    |  9 -  0 -  7 |  56% |  2 |  0
    L.A. Chargers  |  9 -  0 -  7 |  56% |  0 |  2
    Oakland        |  8 -  0 -  8 |  50% |  0 |  0
    Denver         |  5 -  0 - 11 |  31% | -2 | -8
  AFC South
    Jacksonville   | 13 -  0 -  3 |  81% |  4 |  8
    Tennessee      | 12 -  0 -  4 |  75% |  4 |  6
    Houston        |  5 -  0 - 11 |  31% | -4 | -2
    Indianapolis   |  3 -  0 - 13 |  19% | -4 | -10
NFC
  NFC East
    Philadelphia   |  8 -  0 -  8 |  50% |  2 | -2
    Washington     |  8 -  0 -  8 |  50% |  0 |  2
    Dallas      

In [8]:
season_2018.simulate()

In [20]:
team_values = {team: 0.0 for team in season_2018.teams.values()}

In [21]:
from sportsball.nfl import GameRound

In [22]:
value_map = {
    GameRound.REG.value: {
        1: 1.4,
        0: 0.9,
        -1: 0.4,
    },
    GameRound.WILDCARD.value: {
        1: 0,
        -1: 2.5,
    },
    GameRound.DIVISIONAL.value: {
        1: 5.5,
        -1: 5.5,
    },
    GameRound.CONFERENCE.value: {
        1: 4.0,
        -1: 4.0,
    },
    GameRound.SUPERBOWL.value: {
        1: 13.0,
        -1: 5.0,
    }
}

In [122]:
team_values = {team: 0.0 for team in season_2018.teams.values()}
season_2018.simulate()
for game in season_2018.games:
    away_team = game.away_team
    home_team = game.home_team
    team_values[away_team] += value_map[game.game_round.value][
        away_team._mapgmresult(game)]
    team_values[home_team] += value_map[game.game_round.value][
        home_team._mapgmresult(game)]
for game in season_2018.playoff_games:
    away_team = game.away_team
    home_team = game.home_team
    team_values[away_team] += value_map[game.game_round.value][
        away_team._mapgmresult(game)]
    team_values[home_team] += value_map[game.game_round.value][
        home_team._mapgmresult(game)]

In [33]:
season_2018.playoff_games

{        | Week 18 | bears      at PACKERS    | bears,
         | Week 18 | chiefs     at BENGALS    | bengals,
         | Week 18 | ravens     at TITANS     | titans,
         | Week 18 | saints     at SEAHAWKS   | seahawks,
         | Week 19 | bears      at REDSKINS   | bears,
         | Week 19 | bengals    at BRONCOS    | broncos,
         | Week 19 | seahawks   at BUCCANEERS | buccaneers,
         | Week 19 | titans     at DOLPHINS   | titans,
         | Week 20 | bears      vs buccaneers | bears,
         | Week 20 | broncos    vs titans     | broncos,
         | Week 22 | broncos    vs bears      | broncos}

In [34]:
season_2018.teams['Philadelphia'].print_record()

04 - 00 - 12


In [131]:
season_2018.teams['Philadelphia'].games

[12 - 18 | Week  1 | falcons    at EAGLES     | eagles,
 21 - 27 | Week  2 | eagles     at BUCCANEERS | buccaneers,
 16 - 20 | Week  3 | colts      at EAGLES     | eagles,
         | Week  4 | eagles     at TITANS     | eagles,
         | Week  5 | vikings    at EAGLES     | eagles,
         | Week  6 | eagles     at GIANTS     | eagles,
         | Week  7 | panthers   at EAGLES     | eagles,
         | Week  8 | eagles     at JAGUARS    | jaguars,
         | Week 10 | cowboys    at EAGLES     | eagles,
         | Week 11 | eagles     at SAINTS     | eagles,
         | Week 12 | giants     at EAGLES     | eagles,
         | Week 13 | redskins   at EAGLES     | eagles,
         | Week 14 | eagles     at COWBOYS    | eagles,
         | Week 15 | eagles     at RAMS       | rams,
         | Week 16 | texans     at EAGLES     | eagles,
         | Week 17 | eagles     at REDSKINS   | eagles]

In [124]:
for k, v in sorted(team_values.items(), key=lambda x: x[1], reverse=True):
    print('{:15} | ${:0.2f}'.format(k.name, v))

Baltimore       | $39.90
Detroit         | $29.90
Philadelphia    | $28.90
Indianapolis    | $25.90
L.A. Rams       | $23.90
Denver          | $20.90
Carolina        | $20.90
Tennessee       | $20.90
Atlanta         | $19.90
Jacksonville    | $17.90
New Orleans     | $17.90
Buffalo         | $16.90
L.A. Chargers   | $14.40
Washington      | $14.40
Tampa Bay       | $14.40
New England     | $14.40
Dallas          | $14.40
San Francisco   | $14.40
Green Bay       | $13.90
Minnesota       | $13.90
Miami           | $13.40
N.Y. Jets       | $13.40
Cincinnati      | $13.40
Houston         | $13.40
Kansas City     | $13.40
Chicago         | $13.40
Cleveland       | $12.90
Pittsburgh      | $12.90
Oakland         | $12.40
Arizona         | $11.40
Seattle         | $11.40
N.Y. Giants     | $9.40


In [38]:
season_2018.games[2].prob_win

0.5803379668180191

In [23]:
team_values = {team: 0.0 for team in season_2018.teams.values()}
for ii in range(4000):
    season_2018.simulate()
    for game in season_2018.games:
        away_team = game.away_team
        home_team = game.home_team
        team_values[away_team] += value_map[game.game_round.value][
            away_team._mapgmresult(game)]
        team_values[home_team] += value_map[game.game_round.value][
            home_team._mapgmresult(game)]
    for game in season_2018.playoff_games:
        away_team = game.away_team
        home_team = game.home_team
        team_values[away_team] += value_map[game.game_round.value][
            away_team._mapgmresult(game)]
        team_values[home_team] += value_map[game.game_round.value][
            home_team._mapgmresult(game)]

In [24]:
for k, v in sorted(team_values.items(), key=lambda x: x[1], reverse=True):
    print('{:15} | ${:0.2f}'.format(k.name, v / 4000.0))

L.A. Rams       | $28.50
Kansas City     | $25.85
New Orleans     | $22.88
New England     | $22.12
Jacksonville    | $21.23
Baltimore       | $20.40
Cincinnati      | $19.98
L.A. Chargers   | $19.32
Green Bay       | $19.28
Chicago         | $18.52
Philadelphia    | $18.37
Minnesota       | $17.91
Carolina        | $17.85
Tennessee       | $17.48
Miami           | $17.38
Pittsburgh      | $16.62
Atlanta         | $16.16
Dallas          | $16.13
Washington      | $16.03
Houston         | $15.79
Tampa Bay       | $15.15
Seattle         | $15.14
Denver          | $14.56
San Francisco   | $14.10
N.Y. Giants     | $13.34
N.Y. Jets       | $13.26
Indianapolis    | $13.26
Detroit         | $13.24
Cleveland       | $13.17
Oakland         | $13.00
Buffalo         | $11.98
Arizona         | $10.79


In [148]:
sum([v / 4000 for k, v in team_values.items()])

548.8000000002513

In [16]:
_team_values = {team: 0.0 for team in season_2018.teams.values()}
for ii in range(2000):
    season_2018.simulate()
    for game in season_2018.games:
        away_team = game.away_team
        home_team = game.home_team
        _team_values[away_team] += value_map[game.game_round.value][
            away_team._mapgmresult(game)]
        _team_values[home_team] += value_map[game.game_round.value][
            home_team._mapgmresult(game)]
    for game in season_2018.playoff_games:
        away_team = game.away_team
        home_team = game.home_team
        _team_values[away_team] += value_map[game.game_round.value][
            away_team._mapgmresult(game)]
        _team_values[home_team] += value_map[game.game_round.value][
            home_team._mapgmresult(game)]

In [22]:
for k, v in sorted(_team_values.items(), key=lambda x: x[1], reverse=True):
    print('{:15} | ${:0.2f}'.format(k.name, v / 2000.0))

L.A. Rams       | $24.10
Kansas City     | $21.59
Jacksonville    | $19.93
Baltimore       | $19.49
Philadelphia    | $19.31
New Orleans     | $18.59
New England     | $18.51
Miami           | $18.19
L.A. Chargers   | $18.11
Chicago         | $17.69
Tampa Bay       | $17.60
Carolina        | $17.57
Cincinnati      | $17.32
Atlanta         | $17.02
Tennessee       | $16.86
Minnesota       | $16.83
Pittsburgh      | $16.29
Washington      | $15.90
Detroit         | $15.85
Seattle         | $15.73
Dallas          | $15.31
San Francisco   | $15.18
Cleveland       | $15.08
Indianapolis    | $14.98
Buffalo         | $14.98
Denver          | $14.76
N.Y. Jets       | $14.70
Green Bay       | $14.65
N.Y. Giants     | $13.81
Houston         | $13.23
Oakland         | $12.57
Arizona         | $12.05


In [20]:
sum([v / 2000 for k, v in _team_values.items()])

533.8000000003052

In [74]:
games = season_2018.sim_playoffs()

In [76]:
game = games.pop()

In [67]:
team_to_seed = {
    team: seed
    for cnfrnce_str, cnfrnce in playoff_seeds.items()
    for seed, team in cnfrnce.items()
}

In [69]:
afc_seeds, nfc_seeds = playoff_seeds.values()

In [54]:
def sim_playoffs(afc_seeds, nfc_seeds, season):
    # Round 1
    # afc
    # 5 vs. 4 (home)
    game1 = Game(
        week=18,
        season=season,
        neutral=False,
        playoff=True,
        away_team=afc_seeds[5],
        home_team=afc_seeds[4],
        away_score=None,
        home_score=None)

    # 6 vs. 3 (home)
    game2 = Game(
        week=18,
        season=season,
        neutral=False,
        playoff=True,
        away_team=afc_seeds[6],
        home_team=afc_seeds[3],
        away_score=None,
        home_score=None)

    # nfc
    # 5 vs. 4 (home)
    game3 = Game(
        week=18,
        season=season,
        neutral=False,
        playoff=True,
        away_team=nfc_seeds[5],
        home_team=nfc_seeds[4],
        away_score=None,
        home_score=None)

    # 6 vs. 3 (home)
    game4 = Game(
        week=18,
        season=season,
        neutral=False,
        playoff=True,
        away_team=nfc_seeds[6],
        home_team=nfc_seeds[3],
        away_score=None,
        home_score=None)

    games = {
        game1,
        game2,
        game3,
        game4,
    }

    for game in games:
        home_adv = season_2018.get_home_adv(system='vegas', week=18)
        game.prob_win = game.predict_prob_vegas(0.1354771, home_adv=home_adv)
        game.simulate()

    winners = {game.winner() for game in games}

    afc_hiseed, afc_lowseed = sorted(
        {k
         for k, v in afc_seeds.items() if v in winners})
    nfc_hiseed, nfc_lowseed = sorted(
        {k
         for k, v in nfc_seeds.items() if v in winners})

    # Divisional Round
    # afc
    # lowest vs. 1 (home)
    game5 = Game(
        week=19,
        season=season,
        neutral=False,
        playoff=True,
        away_team=afc_seeds[afc_lowseed],
        home_team=afc_seeds[1],
        away_score=None,
        home_score=None)

    # 6 vs. 3 (home)
    game6 = Game(
        week=19,
        season=season,
        neutral=False,
        playoff=True,
        away_team=afc_seeds[afc_hiseed],
        home_team=afc_seeds[2],
        away_score=None,
        home_score=None)

    # nfc
    # 5 vs. 4 (home)
    game7 = Game(
        week=19,
        season=season,
        neutral=False,
        playoff=True,
        away_team=nfc_seeds[nfc_lowseed],
        home_team=nfc_seeds[1],
        away_score=None,
        home_score=None)

    # 6 vs. 3 (home)
    game8 = Game(
        week=19,
        season=season,
        neutral=False,
        playoff=True,
        away_team=nfc_seeds[nfc_hiseed],
        home_team=nfc_seeds[2],
        away_score=None,
        home_score=None)

    games = {
        game5,
        game6,
        game7,
        game8,
    }

    for game in games:
        home_adv = season_2018.get_home_adv(system='vegas', week=19)
        game.prob_win = game.predict_prob_vegas(0.1354771, home_adv=home_adv)
        game.simulate()

    # Conference Championships
    game9 = Game(
        week=20,
        season=season,
        neutral=True,
        playoff=True,
        away_team=game5.winner(),
        home_team=game6.winner(),
        away_score=None,
        home_score=None)

    game10 = Game(
        week=20,
        season=season,
        neutral=True,
        playoff=True,
        away_team=game7.winner(),
        home_team=game8.winner(),
        away_score=None,
        home_score=None)

    games = {
        game9,
        game10,
    }

    for game in games:
        home_adv = season_2018.get_home_adv(system='vegas', week=19)
        game.prob_win = game.predict_prob_vegas(0.1354771, home_adv=home_adv)
        game.simulate()

    # Superbowl
    game11 = Game(
        week=22,
        season=season,
        neutral=True,
        playoff=True,
        away_team=game9.winner(),
        home_team=game10.winner(),
        away_score=None,
        home_score=None)

    home_adv = season_2018.get_home_adv(system='vegas', week=19)
    game11.prob_win = game11.predict_prob_vegas(0.1354771, home_adv=home_adv)
    game11.simulate()

    return {
        game1, game2, game3, game4, game5, game6, game7, game8, game9, game10,
        game11
    }

In [ ]:
for game in (game1:
    away_team = game.away_team
    home_team = game.home_team
    away_team.games.append(game)
    home_team.games.append(game)

In [41]:
teammap['AFC']['cnfrnce_seeds']

{1: patriots: New England,
 2: chargers: L.A. Chargers,
 3: steelers: Pittsburgh,
 4: texans: Houston,
 5: dolphins: Miami,
 6: broncos: Denver}

In [62]:
winners = sim_playoffs(
    teammap['AFC']['cnfrnce_seeds'],
    teammap['NFC']['cnfrnce_seeds'],
    season=2018)

In [63]:
winners

{        | Week 18 | broncos    at STEELERS   | steelers,
         | Week 18 | dolphins   at TEXANS     | texans,
         | Week 18 | falcons    at REDSKINS   | redskins,
         | Week 18 | saints     at VIKINGS    | vikings,
         | Week 19 | redskins   at RAMS       | redskins,
         | Week 19 | steelers   at CHARGERS   | chargers,
         | Week 19 | texans     at PATRIOTS   | patriots,
         | Week 19 | vikings    at BUCCANEERS | vikings,
         | Week 20 | patriots   vs chargers   | patriots,
         | Week 20 | redskins   vs vikings    | vikings,
         | Week 22 | patriots   vs vikings    | patriots}

In [53]:
{winner.division for winner in winners}

{'AFC East', 'AFC North', 'NFC East', 'NFC North'}

In [32]:
for game in sim_playoffs(
        teammap['AFC']['cnfrnce_seeds'],
        teammap['NFC']['cnfrnce_seeds'],
        season=2018):
    home_adv = season_2018.get_home_adv(system='vegas', week=18)
    game.prob_win = game.predict_prob_vegas(0.1354771, home_adv=home_adv)
    game.simulate()
    print(game)

        | Week 18 | dolphins   at TEXANS     | texans
        | Week 18 | broncos    at STEELERS   | broncos
        | Week 18 | falcons    at REDSKINS   | falcons
        | Week 18 | saints     at VIKINGS    | vikings


In [30]:
import functools

In [248]:
@functools.total_ordering
class MyNum(int):
            
    def __lt__(self, other, switch=True):
        if switch:
            return int(self) > int(other)
        return int(self) < int(other)

In [256]:
numA = MyNum(3)
numB = MyNum(3)
numC = MyNum(5)

In [258]:
numA is numB

False

In [147]:
def quick_sort(items, switch=None):
    """ Implementation of quick sort """
    if len(items) > 1:
        pivot_index = int(len(items) / 2)
        smaller_items = []
        larger_items = []

        for i, val in enumerate(items):
            if i != pivot_index:
                if val.__lt__(items[pivot_index], switch=switch):
                    smaller_items.append(val)
                else:
                    larger_items.append(val)

        return quick_sort(
            smaller_items, switch=switch) + [items[pivot_index]] + quick_sort(
                larger_items, switch=switch)
    else:
        return items

In [179]:
for cnfrnce_str, cnfrnce in season_2018.league.items():
    print(cnfrnce_str)
    for dvsn_str, dvsn in cnfrnce.items():
        print('  {}'.format(dvsn_str))
        for team in sorted(dvsn, reverse=True):
            print('    {}'.format('{:15}| {:2} - {:2} - {:2} | {:>4}'.format(
                team.name,
                team.wins,
                team.ties,
                team.losses,
                '{:.2%}'.format(team.winpct),
            )))

AFC
  AFC East
    New England    | 12 -  0 -  4 | 75.00%
    N.Y. Jets      |  8 -  0 -  8 | 50.00%
    Miami          |  7 -  0 -  9 | 43.75%
    Buffalo        |  6 -  0 - 10 | 37.50%
  AFC North
    Baltimore      | 12 -  0 -  4 | 75.00%
    Pittsburgh     |  8 -  1 -  7 | 53.12%
    Cincinnati     |  8 -  0 -  8 | 50.00%
    Cleveland      |  6 -  1 -  9 | 40.62%
  AFC West
    Kansas City    | 10 -  0 -  6 | 62.50%
    Denver         |  9 -  0 -  7 | 56.25%
    L.A. Chargers  |  8 -  0 -  8 | 50.00%
    Oakland        |  4 -  0 - 12 | 25.00%
  AFC South
    Jacksonville   |  9 -  0 -  7 | 56.25%
    Tennessee      |  7 -  0 -  9 | 43.75%
    Indianapolis   |  7 -  0 -  9 | 43.75%
    Houston        |  7 -  0 -  9 | 43.75%
NFC
  NFC East
    Philadelphia   |  9 -  0 -  7 | 56.25%
    N.Y. Giants    |  9 -  0 -  7 | 56.25%
    Dallas         |  7 -  0 -  9 | 43.75%
    Washington     |  6 -  0 - 10 | 37.50%
  NFC North
    Minnesota      |  9 -  1 -  6 | 59.38%
    Chicago        |

In [133]:
dvsn = season_2018.league['NFC']['NFC North']

In [129]:
def division_winner(dvsn):
    bestrecord = max([team.wins + 0.5 * team.ties for team in dvsn])

    division_champs = [
        team for team in dvsn if team.wins + 0.5 * team.ties == bestrecord
    ]
    if len(division_champs) == 1:
        return division_champs[0]
    return divisional_tiebreak(division_champs)


def divisional_tiebreak(teams):
    if len(teams) == 1:
        return teams[0]
    _teams = head_to_head(teams)
    if len(_teams) == 1:
        print('tie broken via head-to-head record')
        return _teams[0]
    nteams = len(_teams)
    _teams = division_records(_teams)
    if len(_teams) == 1:
        print('tie broken via divisional record')
        return _teams[0]
    if len(_teams) != nteams:
        return divisional_tiebreak(_teams)
    # now just return random team
    print('tie broken via coin toss')
    return numpy.random.choice(_teams, size=1)[0]


def head_to_head(teams):
    cntnr = {
        team: wins(
            Counter([
                team._mapgmresult(game) for game in team.games
                if game.away_team in teams and game.home_team in teams
            ]))
        for team in teams
    }
    bestrecord = max(list(cntnr.values()))
    return [
        team for team, wins in cntnr.items()
        if wins == max(list(cntnr.values()))
    ]


def wins(counter):
    return float(counter[1] + 0.5 * counter[0])


def division_records(teams):
    cntnr = {
        team: wins(
            Counter([
                team._mapgmresult(game) for game in team.games
                if game.away_team.division == game.home_team.division
            ]))
        for team in teams
    }
    bestrecord = max(list(cntnr.values()))
    return [
        team for team, wins in cntnr.items()
        if wins == max(list(cntnr.values()))
    ]

In [149]:
def conference_wildcard(conference):
    bestrecord = max([team.wins + 0.5 * team.ties for team in conference])

    conference_champs = [
        team for team in conference
        if team.wins + 0.5 * team.ties == bestrecord
    ]
    if len(conference_champs) == 1:
        return conference_champs[0]
    return conference_champs

In [ ]:
division_winner(dvsn)

In [157]:
division_winners = {
    cnfrnce_str:
    {dvsn_str: division_winner(dvsn)
     for dvsn_str, dvsn in cnfrnce.items()}
    for cnfrnce_str, cnfrnce in season_2018.league.items()
}

In [163]:
sorted(list(division_winners['NFC'].values()))

[eagles: Philadelphia, packers: Green Bay, panthers: Carolina, rams: L.A. Rams]

In [161]:
for team in division_winners['NFC'].values():
    print('    {}'.format('{:15}| {:2} - {:2} - {:2} | {:>4}'.format(
        team.name,
        team.wins,
        team.ties,
        team.losses,
        '{:.2%}'.format(team.winpct),
    )))

    Philadelphia   | 11 -  0 -  5 | 68.75%
    Green Bay      | 12 -  1 -  3 | 78.12%
    Carolina       | 12 -  0 -  4 | 75.00%
    L.A. Rams      | 13 -  0 -  3 | 81.25%


In [146]:
teams_left = [team for team in season_2018.teams.values() if team not in division_winners]

In [152]:
conference = season_2018.league['NFC']

In [154]:
conference_wildcard([team for team in teams_left if team.conference == 'AFC'])

chargers: L.A. Chargers

In [139]:
for cnfrnce_str, cnfrnce in season_2018.league.items():
    print(cnfrnce_str)
    for dvsn_str, dvsn in cnfrnce.items():
        print('  {}'.format(dvsn_str))

        team = division_winner(dvsn)
        print('    {}'.format('{:15}| {:2} - {:2} - {:2} | {:>4}'.format(
            team.name,
            team.wins,
            team.ties,
            team.losses,
            '{:.2%}'.format(team.winpct),
        )))

AFC
  AFC East
    New England    |  9 -  0 -  7 | 56.25%
  AFC North
    Baltimore      | 12 -  0 -  4 | 75.00%
  AFC West
    Kansas City    | 12 -  0 -  4 | 75.00%
  AFC South
    Indianapolis   | 11 -  0 -  5 | 68.75%
NFC
  NFC East
    Philadelphia   | 11 -  0 -  5 | 68.75%
  NFC North
    Green Bay      | 12 -  1 -  3 | 78.12%
  NFC South
    Carolina       | 12 -  0 -  4 | 75.00%
  NFC West
    L.A. Rams      | 13 -  0 -  3 | 81.25%


In [14]:
season_2018.attach_sagarin_ratings()
season_2018.attach_538_ratings()
season_2018.attach_massey_ratings()
season_2018.attach_scorex_ratings()
season_2018.attach_vegas_ratings()

In [15]:
season_2018.set_game_probs('vegas')

In [16]:
season_2018.simulate()

In [344]:
season_2018.teams['Miami'].losses

8

In [308]:
df['pct'] = df.wins / (df.wins + df.losses)

In [312]:
df.sort_values(
    ['division', 'pct'], ascending=[True, False])[[
        'name',
        'conference',
        'division',
        'wins',
        'losses',
        'pct',
    ]]

,name,conference,division,wins,losses,pct
1,Miami,AFC,AFC East,10,6,0.6250
2,N.Y. Jets,AFC,AFC East,10,6,0.6250
3,New England,AFC,AFC East,10,6,0.6250
0,Buffalo,AFC,AFC East,1,15,0.0625
4,Baltimore,AFC,AFC North,11,5,0.6875
5,Cincinnati,AFC,AFC North,8,8,0.5000
7,Pittsburgh,AFC,AFC North,7,9,0.4375
6,Cleveland,AFC,AFC North,6,10,0.3750
11,Jacksonville,AFC,AFC South,13,3,0.8125
15,Tennessee,AFC,AFC South,9,7,0.5625


In [1]:
import pandas
import sklearn

from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, log_loss

In [168]:
from sklearn.model_selection import GroupKFold

In [148]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [8]:
dtf = pandas.read_csv('./../data/processed/nfl_data.csv')
dtf = dtf.loc[(dtf.home_win == 0) | (dtf.home_win == 1)]

In [239]:
dtf

,season,five38_home_rating,five38_away_rating,massey_home_rating,massey_away_rating,sagarin_home_rating,sagarin_away_rating,home_win,gamenum,home_adv
0,2017,1434,1559,24.83,22.80,17.12,20.50,1.0,1,1
1,2017,1529,1600,21.28,18.56,23.01,21.80,1.0,2,1
2,2017,1476,1497,21.00,24.19,20.23,18.40,1.0,3,1
3,2017,1498,1480,21.69,11.05,24.08,17.84,1.0,4,1
4,2017,1623,1489,32.83,15.50,27.78,19.01,1.0,5,1
5,2017,1489,1387,29.52,23.75,24.02,11.43,0.0,6,1
6,2017,1455,1514,17.59,25.61,19.77,21.57,0.0,7,1
7,2017,1598,1436,31.03,18.36,27.66,19.74,1.0,8,1
8,2017,1282,1460,20.29,22.22,12.33,14.98,0.0,9,1
9,2017,1608,1477,24.66,21.94,26.23,20.71,0.0,10,1


In [144]:
pipe = sklearn.pipeline.Pipeline([
    ('featurize',
     DataFrameMapper(
         [
             (['five38_home_rating'], StandardScaler()),
             (['five38_away_rating'], StandardScaler()),
             (['massey_home_rating'], StandardScaler()),
             (['massey_away_rating'], StandardScaler()),
             (['sagarin_home_rating'], StandardScaler()),
             (['sagarin_away_rating'], StandardScaler()),
         ],
         sparse=True)),
    ('lm',
     SGDClassifier(
         alpha=0.08,
         loss='log',
         fit_intercept=False,
         penalty='l2',
         max_iter=1000,
         tol=1e-3,
     )),
])

In [230]:
pipe = sklearn.pipeline.Pipeline([
    ('featurize',
     DataFrameMapper(
         [
             ([
                 'five38_away_rating',
                 'five38_home_rating',
                 'massey_away_rating',
                 'massey_home_rating',
                 'sagarin_away_rating',
                 'sagarin_home_rating',
             ], StandardScaler()),
         ],
         sparse=True)),
    ('lm', LogisticRegression(C=0.4)),
])

In [232]:
x = pipe.named_steps['featurize']

In [236]:
x.features[0][1].scale_

array([97.86976773, 97.86976773,  4.51784368,  4.51784368,  4.5378788 ,
        4.5378788 ])

In [237]:
x.features[0][1].mean_

array([1513.71610169, 1513.71610169,   23.00144068,   23.00144068,
         21.25025424,   21.25025424])

In [ ]:
cv = GroupKFold(n_splits=20)

In [206]:
xxx = cross_val_score(
    pipe,
    X=dtf,
    y=dtf.home_win,
    groups=dtf.gamenum,
    scoring=make_scorer(
        log_loss,
        greater_is_better=False,
        needs_proba=True,
    ),
    cv=cv.split(X=dtf, groups=dtf.gamenum),
    n_jobs=-1,
)

In [207]:
xxx.mean()

-0.5844360835663382

In [ ]:
clf = GridSearchCV(svc, parameters, scor)

In [231]:
pipe.fit(X=dtf, y=dtf.home_win)

Pipeline(memory=None,
     steps=[('featurize', DataFrameMapper(default=False, df_out=False,
        features=[(['five38_away_rating', 'five38_home_rating', 'massey_away_rating', 'massey_home_rating', 'sagarin_away_rating', 'sagarin_home_rating'], StandardScaler(copy=True, with_mean=True, with_std=True))],
        input_df=Fa...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [220]:
lm = pipe.named_steps['lm']

In [235]:
lm.coef_

array([[-0.48811602,  0.48811602,  0.83260993, -0.83260993,  0.17701139,
        -0.17701139]])

In [238]:
0.48811602/97.86976773

0.004987403478330498